Primero creamos una serie de funciones axiliares para el procesado final

In [170]:
import pandas as pd
import re

def fusionar_datos(ruta_corpus1 = "Corpus.csv", ruta_corpus2 = "Corpus_mirabel.csv", ruta_corpusFinal="Corpus.csv"):
    corpus1 = pd.read_csv(ruta_corpus1, lineterminator='\n')
    corpus2 = pd.read_csv(ruta_corpus2, lineterminator='\n')
    
    corpus = pd.concat([corpus1, corpus2], axis=0, ignore_index=True)
    corpus.to_csv(ruta_corpusFinal, sep=',', index=False)
    return corpus
    
def eliminar_nulos(corpus):
    corpus = corpus.dropna(how='any',axis=0)
    return corpus

def subcadena_contenida(s, izq, der):
    try:
        return s[s.index(izq):s.index(der)+1]
    except:
        return None

def encontrar_pregunta(s):
    izq = '¿'
    der = '?'
    return subcadena_contenida(s, izq, der)

def comenzar_mayuscula(s):
    if len(s) > 1:
        return s[0].upper()+s[1:]
    else:
        return None

def eliminar_fallo_prompt(s):
    aux = s.split(':')
    if len(aux) > 1:
        return aux[1][1:]
    else:
        return aux[0]

def formatear_preguntas(corpus):
    for index, row in corpus.iterrows():
        print(f"Fase {index} de {corpus.shape[0]}", end="\r")
        
        pregunta = encontrar_pregunta(row["Pregunta"])
        if pregunta is not None:
            corpus.loc[index,'Pregunta'] = pregunta
        else:
            corpus.drop(index, axis=0, inplace=True)

def formatear_respuestas(corpus):
    for index, row in corpus.iterrows():
        print(f"Fase {index} de {corpus.shape[0]}", end="\r")
        
        respuesta = row["Respuesta"]
        if respuesta is not None:
            respuesta = comenzar_mayuscula(eliminar_fallo_prompt(respuesta))
            corpus.loc[index,'Respuesta'] = respuesta
        else:
            corpus.drop(index, axis=0, inplace=True)
            
def reiniciar_indice(corpus):
    return corpus.reset_index(drop=True)

def eliminar_repetidos(corpus, subset=None):
    if subset is not None:
        return corpus.drop_duplicates(subset=subset)
    else:
        return corpus.drop_duplicates()

Comenzamos aplicando los métodos para limpiar los datos para los modelos de clasificación y evaluación

In [175]:
corpus = fusionar_datos()

In [176]:
corpus.shape

(54119, 5)

In [177]:
corpus = eliminar_nulos(corpus)

In [178]:
corpus.shape

(43433, 5)

In [179]:
corpus.head()

,Contexto,Documento,Pregunta,Respuesta,Modelo
0,Protección Menor en lo referido a actuación co...,000ProtocoloPrevencionSuicidio3 Navarra.pdf,¿Cuál es el propósito de la protección al menor?,El propósito de la protección al menor es pro...,bertin-project/bertin-gpt-j-6B-alpaca
1,Una cuestión clave es realizar una historia cl...,000ProtocoloPrevencionSuicidio3 Navarra.pdf,¿Qué medidas puede tomar el médico para ayuda...,El médico puede proporcionar orientación y ap...,bertin-project/bertin-gpt-j-6B-alpaca
2,La Constitución Española20 reconoce y protege ...,000ProtocoloPrevencionSuicidio3 Navarra.pdf,gunta: ¿Cuál es el propósito de la cláusula de...,puesta: La cláusula de conciencia es un derech...,bertin-project/bertin-gpt-j-6B-alpaca
3,Directamente relacionado con la intimidad está...,000ProtocoloPrevencionSuicidio3 Navarra.pdf,gunta: ¿Cuál es el derecho a la intimidad?,puesta: El derecho a la intimidad es el derech...,bertin-project/bertin-gpt-j-6B-alpaca
4,"Boletín Oficial del Estado, Nº 102, (29 de abr...",000ProtocoloPrevencionSuicidio3 Navarra.pdf,gunta: ¿Cuál es la Constitución de España?,puesta: La Constitución de España es la Ley Su...,bertin-project/bertin-gpt-j-6B-alpaca


In [180]:
formatear_preguntas(corpus)
corpus.head()

,Contexto,Documento,Pregunta,Respuesta,Modelo
0,Protección Menor en lo referido a actuación co...,000ProtocoloPrevencionSuicidio3 Navarra.pdf,¿Cuál es el propósito de la protección al menor?,El propósito de la protección al menor es pro...,bertin-project/bertin-gpt-j-6B-alpaca
1,Una cuestión clave es realizar una historia cl...,000ProtocoloPrevencionSuicidio3 Navarra.pdf,¿Qué medidas puede tomar el médico para ayudar...,El médico puede proporcionar orientación y ap...,bertin-project/bertin-gpt-j-6B-alpaca
2,La Constitución Española20 reconoce y protege ...,000ProtocoloPrevencionSuicidio3 Navarra.pdf,¿Cuál es el propósito de la cláusula de concie...,puesta: La cláusula de conciencia es un derech...,bertin-project/bertin-gpt-j-6B-alpaca
3,Directamente relacionado con la intimidad está...,000ProtocoloPrevencionSuicidio3 Navarra.pdf,¿Cuál es el derecho a la intimidad?,puesta: El derecho a la intimidad es el derech...,bertin-project/bertin-gpt-j-6B-alpaca
4,"Boletín Oficial del Estado, Nº 102, (29 de abr...",000ProtocoloPrevencionSuicidio3 Navarra.pdf,¿Cuál es la Constitución de España?,puesta: La Constitución de España es la Ley Su...,bertin-project/bertin-gpt-j-6B-alpaca


In [181]:
formatear_respuestas(corpus)
corpus.head()

,Contexto,Documento,Pregunta,Respuesta,Modelo
0,Protección Menor en lo referido a actuación co...,000ProtocoloPrevencionSuicidio3 Navarra.pdf,¿Cuál es el propósito de la protección al menor?,El propósito de la protección al menor es pro...,bertin-project/bertin-gpt-j-6B-alpaca
1,Una cuestión clave es realizar una historia cl...,000ProtocoloPrevencionSuicidio3 Navarra.pdf,¿Qué medidas puede tomar el médico para ayudar...,El médico puede proporcionar orientación y ap...,bertin-project/bertin-gpt-j-6B-alpaca
2,La Constitución Española20 reconoce y protege ...,000ProtocoloPrevencionSuicidio3 Navarra.pdf,¿Cuál es el propósito de la cláusula de concie...,La cláusula de conciencia es un derecho fundam...,bertin-project/bertin-gpt-j-6B-alpaca
3,Directamente relacionado con la intimidad está...,000ProtocoloPrevencionSuicidio3 Navarra.pdf,¿Cuál es el derecho a la intimidad?,El derecho a la intimidad es el derecho de tod...,bertin-project/bertin-gpt-j-6B-alpaca
4,"Boletín Oficial del Estado, Nº 102, (29 de abr...",000ProtocoloPrevencionSuicidio3 Navarra.pdf,¿Cuál es la Constitución de España?,La Constitución de España es la Ley Suprema de...,bertin-project/bertin-gpt-j-6B-alpaca


In [182]:
corpus.shape

(32489, 5)

In [183]:
corpus=reiniciar_indice(corpus)
corpus.tail()

,Contexto,Documento,Pregunta,Respuesta,Modelo
32484,NO SÍ 15 T8 ¿CODIFICÓ SÍ EN T5 O CODIFICÓ NO ...,mini-kid-entrevista-estructurada-ninos-y-adole...,¿Has estado preocupado mucho o has estado nerv...,"CODIFICAR SÍ, SI LA ANSIEDAD DEL P",clibrain/Llama-2-7b-ft-instruct-es
32485,FUNDACIÓN SI ESTÁS PENSANDO EN SUICIDARTE… LEE...,si-piensas-en-suicidarte-lee-esto.pdf,¿Qué puedo hacer para ayudarme a mí mismo?,"1. Si estás pensando en suicidarte, por favor...",clibrain/Llama-2-7b-ft-instruct-es
32486,Basta con hablar de cómo llegaste hasta este p...,si-piensas-en-suicidarte-lee-esto.pdf,¿Cuál es el regalo que te vas a dar?,Un recurso para enfrentar el,clibrain/Llama-2-7b-ft-instruct-es
32487,"A pesar de sus buenas intenciones, quizás aume...",si-piensas-en-suicidarte-lee-esto.pdf,¿Qué puedes hacer para ayudarte a sentirte mejor?,1. Toma un descanso. 2. Haz ejercicio. 3. Due...,clibrain/Llama-2-7b-ft-instruct-es
32488,Comienza analizando estas palabras: “El suicid...,si-piensas-en-suicidarte-lee-esto.pdf,"¿Cuál es el significado de ""El suicidio no es ...",El hecho de que pienses en suicidarte no quie...,clibrain/Llama-2-7b-ft-instruct-es


In [184]:
corpus = reiniciar_indice(eliminar_repetidos(corpus))
corpus.shape

(22920, 5)

In [185]:
preguntas = corpus["Pregunta"]

In [186]:
preguntas

0         ¿Cuál es el propósito de la protección al menor?
1        ¿Qué medidas puede tomar el médico para ayudar...
2        ¿Cuál es el propósito de la cláusula de concie...
3                      ¿Cuál es el derecho a la intimidad?
4                      ¿Cuál es la Constitución de España?
                               ...                        
22915    ¿Has estado preocupado mucho o has estado nerv...
22916           ¿Qué puedo hacer para ayudarme a mí mismo?
22917                 ¿Cuál es el regalo que te vas a dar?
22918    ¿Qué puedes hacer para ayudarte a sentirte mejor?
22919    ¿Cuál es el significado de "El suicidio no es ...
Name: Pregunta, Length: 22920, dtype: object

In [187]:
preguntas.to_csv('Corpus preguntas.csv',index=False)

In [188]:
corpus.to_csv("Corpus.csv", index=False)